In [47]:
from pprint import pprint

If we think we need a class with multiple inheritance, we should consider writing a **mix-in** class instead. 

Defintion: a **mix-in** is a small class that only defines a set of additional methods that a class should provided. But **mix-in** classes **do not define their own instance attributes** and **do not require a __init__ constructor** to be called.

In [79]:
class ToDictMixin(object):
    
    def to_dict(self):
        print('Entering to_dict')
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        print("-> Entering _traverse_dict with: \n --> instance_dict: ")
        pprint(instance_dict)
        output = {}
        for key, value in instance_dict.items():
            print("key/value is {0}/{1}".format(key, value))
            output[key] = self._traverse(key, value)
            print("\n")
        return output

    def _traverse(self, key, value):

        # as long as the value is an instance of ToDictMixin, we keep looping
        if isinstance(value, ToDictMixin):
            print("----> isinstance(value, ToDictMixin)")
            return value.to_dict()
        
        # if the value is a simple dictionary
        elif isinstance(value, dict):
            print("----> isinstance(value, dict)")
            return self._traverse_dict(value)

        elif isinstance(value, list):
            print("----> isinstance(value, list)")
            return [self._traverse(key, i) for i in value]

        elif hasattr(value, '__dict__'):
            print("---> hasattr(value, '__dict__')")
            return self._traverse_dict(value.__dict__)

        else:
            print("----> else")
            return value

In [80]:
class BinaryTree(ToDictMixin):
    
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.right = right
        self.left = left

In [81]:
tree = BinaryTree(10, left=BinaryTree(7, right=BinaryTree(9)), right=BinaryTree(13, left=BinaryTree(11)))
print(type(tree))

<class '__main__.BinaryTree'>


### Trying to understand the various calls in ToDictMixin

In [63]:
print(tree.__dict__)

{'value': 10, 'left': <__main__.BinaryTree object at 0x1098f63c8>, 'right': <__main__.BinaryTree object at 0x1098f6438>}


In [64]:
print(tree.left.__dict__)

{'value': 7, 'left': None, 'right': <__main__.BinaryTree object at 0x1098f6390>}


tree is a instance of **ToDictMixin** but also **BinaryTree**

In [73]:
isinstance(tree, ToDictMixin)

True

In [75]:
isinstance(tree, BinaryTree)

True

In [65]:
hasattr(tree, '__dict__')

True

type of a **dictionary**

In [78]:
a = {'ma': 10, 'ba': 20}
print(type(a))

<class 'dict'>


In [77]:
isinstance(a, dict)

True

### Running the script

In [82]:
pprint(tree.to_dict())

Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': <__main__.BinaryTree object at 0x1098fb7b8>,
 'right': <__main__.BinaryTree object at 0x1098fb828>,
 'value': 10}
key/value is value/10
----> else


key/value is left/<__main__.BinaryTree object at 0x1098fb7b8>
----> isinstance(value, ToDictMixin)
Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': None,
 'right': <__main__.BinaryTree object at 0x1098fb780>,
 'value': 7}
key/value is value/7
----> else


key/value is left/None
----> else


key/value is right/<__main__.BinaryTree object at 0x1098fb780>
----> isinstance(value, ToDictMixin)
Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': None, 'right': None, 'value': 9}
key/value is value/9
----> else


key/value is left/None
----> else


key/value is right/None
----> else






key/value is right/<__main__.BinaryTree object at 0x1098fb828>
----> isinstance(value, ToDictMixin)
Entering to_dict
-> E

### mix-in routine can be overridden

In [84]:
class BinaryTree(ToDictMixin):
    
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.right = right
        self.left = left
        
class BinaryTreeWithParent(BinaryTree):
    
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
        
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and key == 'parent'):
            return value.value
        else:
            return super()._traverse(key, value)

In [87]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': <__main__.BinaryTreeWithParent object at 0x10850ae10>,
 'parent': None,
 'right': None,
 'value': 10}
key/value is value/10
----> else


key/value is left/<__main__.BinaryTreeWithParent object at 0x10850ae10>
----> isinstance(value, ToDictMixin)
Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': None,
 'parent': <__main__.BinaryTreeWithParent object at 0x1085170b8>,
 'right': <__main__.BinaryTreeWithParent object at 0x108517390>,
 'value': 7}
key/value is value/7
----> else


key/value is left/None
----> else


key/value is right/<__main__.BinaryTreeWithParent object at 0x108517390>
----> isinstance(value, ToDictMixin)
Entering to_dict
-> Entering _traverse_dict with: 
 --> instance_dict: 
{'left': None,
 'parent': <__main__.BinaryTreeWithParent object at 0x10850ae10>,
 'right': None,
 'value': 9}
key/value is value/9
----> else


key/value is left/None
----> else


key/value 